In [17]:
import matplotlib.pyplot as plt
import pymiediff as pmd
import torch
import numpy as np
import h5py
import torch.nn as nn
import torch.optim as optim


In [18]:
starting_wavelength = 380  # nm
ending_wavelength = 750  # nm

N_pt_test = 250

wl = torch.linspace(
    starting_wavelength,
    ending_wavelength,
    N_pt_test,
    dtype=torch.double,
    requires_grad=False,
)

k0 = 2 * torch.pi / wl

In [19]:
# define range of starting parameter combinations
r_c_min, r_c_max = 10.0, 20.0
r_s_min, r_s_max = 45.0, 55.0
n_c_min, n_c_max = 2.0 + 0.1j, 2.0 + 0.1j
n_s_min, n_s_max = 5.0 + 0.2j, 5.0 + 0.2j

# define number of starting parameter combinations
NumComb = 1000

r_c, r_s, n_c, n_s = pmd.seedComb(
    r_c_min,
    r_c_max,
    r_s_min,
    r_s_max,
    n_c_min,
    n_c_max,
    n_s_min,
    n_s_max,
    NumComb=NumComb,
)

In [20]:
# cross_section = np.zeros((NumComb, N_pt_test), dtype=np.float32)

# # Compute cross-section iteratively
# for i in range(NumComb):
#     cross_section[i] = pmd.farfield.cross_sections(
#         k0=k0,
#         r_c=r_c[i],
#         eps_c=n_c[i]**2,
#         r_s=r_s[i],
#         eps_s=n_s[i]**2,
#         eps_env=1,
#     )['q_sca']
#     if i % 50 == 0:
#         print(f"{i}/{NumComb}")

# # Save to HDF5 file
# h5_path = "dataset.h5"
# with h5py.File(h5_path, "w") as f:
#     f.create_dataset("r_c", data=r_c)
#     f.create_dataset("r_s", data=r_s)
#     f.create_dataset("n_c", data=n_c)
#     f.create_dataset("n_s", data=n_s)
#     f.create_dataset("cross_section", data=cross_section)

# print(f"Dataset saved to {h5_path}")

In [21]:
import h5py
import torch
import numpy as np
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

from torch.utils.data import Dataset, DataLoader

class CoreShellDataset(Dataset):
    def __init__(self, h5_file):
        super().__init__()
        self.h5_file = h5_file

        # Open the file to get dataset sizes (but don't keep it open)
        with h5py.File(h5_file, "r") as f:
            self.length = len(f["r_c"])  # Assuming all arrays have the same length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        with h5py.File(self.h5_file, "r") as f:
            r_c = f["r_c"][idx]
            r_s = f["r_s"][idx]
            n_c = f["n_c"][idx]
            n_s = f["n_s"][idx]
            cross_section = f["cross_section"][idx]

        x = torch.tensor([r_c, r_s, n_c, n_s], dtype=torch.float32)
        y = torch.tensor(cross_section, dtype=torch.float32)

        return x, y

# Usage example
h5_path = "dataset.h5"
dataset = CoreShellDataset(h5_path)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [27]:
class MLP(nn.Module):
    def __init__(self, input_dim=250, output_dim=4, hidden_dim=128):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [33]:
def custom_function(a, b, c, d):
    return pmd.farfield.coreshell(
                k0=k0,
                r_c=a,
                eps_c=c**2,
                r_s=b,
                eps_s=d**2,
                eps_env=1,
                )['q_sca']

def train_model(model, dataloader, num_epochs=20, lr=1e-3, device="cuda" if torch.cuda.is_available() else "cpu"):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        total_loss = 0.0
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            # Inverse model
            x_pred = model(y_batch)
            print(x_pred.shape)
            # Forward model
            y_pred = torch.stack([custom_function(*row) for row in x_pred])



            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")

    print("Training complete!")

In [24]:
from torch.utils.data import DataLoader
h5_path = "dataset.h5"

train_dataset = CoreShellDataset(h5_path)
# scaler = train_dataset.scaler  # Save scaler for later use

# Create DataLoader for batch processing
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [34]:
mlp_model = MLP()
train_model(mlp_model, dataloader, num_epochs=50, lr=1e-3)

torch.Size([32, 4])


C:\Users\okcj1g19\AppData\Local\Temp\ipykernel_25080\2103007229.py:29: ComplexWarning: Casting complex values to real discards the imaginary part
  x = torch.tensor([r_c, r_s, n_c, n_s], dtype=torch.float32)


TypeError: 'module' object is not callable

In [ ]:
# Get a sample from the dataset
x_sample, y_sample = train_dataset[0]  # Example input
x_sample = x_sample.unsqueeze(0)  # Add batch dimension

# Run inference
mlp_model.eval()
with torch.no_grad():
    y_pred = mlp_model(x_sample)

# Convert prediction back to original scale
y_pred_original = scaler.inverse_transform(y_pred.numpy())
